In [12]:
import pandas as pd
import numpy as np

from pyecharts.charts import Geo
from pyecharts import options as opts
from pyecharts.globals import ChartType, SymbolType

In [ ]:
data = pd.read_csv('data/qd.csv')

In [85]:
district = ['市南区','市北区','李沧区','崂山区','城阳区','黄岛区','即墨市','胶州市','平度市','莱西市']
# hardcode for now
dig_data = [1,4,2,2,1,9,3,1,2,1]

In [99]:
# 需要标注区的名字 
# 背景颜色可以调整一下 
c = (
        Geo()
        .add_schema(maptype="青岛", itemstyle_opts=opts.ItemStyleOpts(color=" #154360 ", border_color="#111"))
        .add(
            "确诊人数-Scatter",
            [list(z) for z in zip(district, dig_data)],
            type_=ChartType.EFFECT_SCATTER,
        )
        .add(
            "确诊人数-heatmap",
            [list(z) for z in zip(district, dig_data)],
            type_=ChartType.HEATMAP,
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(max_=10),
            title_opts=opts.TitleOpts(title='青岛各区市确诊人数'),
        )
    )


c.render_notebook()
c.render()

'C:\\Users\\Flora\\Documents\\python\\project\\qingdao-ncov-viz\\ds\\render.html'

In [51]:
#EFFECT_SCATTER
#HEATMAP

'C:\\Users\\Flora\\Documents\\python\\project\\qingdao-ncov-viz\\ds\\render.html'

In [98]:
c = (
    Map()
    .add("确诊人数", [list(z) for z in zip(district,dig_data)], "青岛",)
    .set_global_opts(title_opts=opts.TitleOpts(title="Map-基本示例"),visualmap_opts=opts.VisualMapOpts(max_=10, is_piecewise=True))
)

c.render_notebook()